# Animation/generating figures

Show how to animate/generating figures using the PyVista API to make awesome online documentation. 

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Marching Cubes
==============

Generate a surface from a scalar field using the flying edges and
marching cubes filters as provided by the `contour
<pyvista.DataSetFilters.contour>` filter.

Special thanks to GitHub user [stla](https://gist.github.com/stla) for
providing examples.


In [ ]:
import numpy as np
import pyvista as pv

Spider Cage
===========

Use the marching cubes algorithm to extract the isosurface generated
from the spider cage function.


In [ ]:
a = 0.9


def spider_cage(x, y, z):
    x2 = x * x
    y2 = y * y
    x2_y2 = x2 + y2
    return (np.sqrt((x2 - y2) ** 2 / x2_y2 + 3 * (z * np.sin(a)) ** 2) - 3) ** 2 + 6 * (
        np.sqrt((x * y) ** 2 / x2_y2 + (z * np.cos(a)) ** 2) - 1.5
    ) ** 2


# create a uniform grid to sample the function with
n = 100
x_min, y_min, z_min = -5, -5, -3
grid = pv.UniformGrid(
    dims=(n, n, n),
    spacing=(abs(x_min) / n * 2, abs(y_min) / n * 2, abs(z_min) / n * 2),
    origin=(x_min, y_min, z_min),
)
x, y, z = grid.points.T

# sample and plot
values = spider_cage(x, y, z)
mesh = grid.contour(1, values, method="marching_cubes", rng=[1, 0])
dist = np.linalg.norm(mesh.points, axis=1)
mesh.plot(
    scalars=dist, smooth_shading=True, specular=5, cmap="plasma", show_scalar_bar=False
)

Barth Sextic
============

Use the flying edges algorithm to extract the isosurface generated from
the Barth sextic function.


In [ ]:
phi = (1 + np.sqrt(5)) / 2
phi2 = phi * phi


def barth_sextic(x, y, z):
    x2 = x * x
    y2 = y * y
    z2 = z * z
    arr = (
        3 * (phi2 * x2 - y2) * (phi2 * y2 - z2) * (phi2 * z2 - x2)
        - (1 + 2 * phi) * (x2 + y2 + z2 - 1) ** 2
    )
    nan_mask = x2 + y2 + z2 > 3.1
    arr[nan_mask] = np.nan
    return arr


# create a uniform grid to sample the function with
n = 100
k = 2.0
x_min, y_min, z_min = -k, -k, -k
grid = pv.UniformGrid(
    dims=(n, n, n),
    spacing=(abs(x_min) / n * 2, abs(y_min) / n * 2, abs(z_min) / n * 2),
    origin=(x_min, y_min, z_min),
)
x, y, z = grid.points.T

# sample and plot
values = barth_sextic(x, y, z)
mesh = grid.contour(1, values, method="flying_edges", rng=[-0.0, 0])
dist = np.linalg.norm(mesh.points, axis=1)
mesh.plot(
    scalars=dist, smooth_shading=True, specular=5, cmap="plasma", show_scalar_bar=False
)

Animate Barth Sextic
====================

Show 15 frames of various isocurves extracted from the Barth sextic
function.


In [ ]:
def angle_to_range(angle):
    return -2 * np.sin(angle)


mesh = grid.contour(1, values, method="flying_edges", rng=[angle_to_range(0), 0])
dist = np.linalg.norm(mesh.points, axis=1)

pl = pv.Plotter()
pl.add_mesh(
    mesh,
    scalars=dist,
    smooth_shading=True,
    specular=5,
    rng=[0.5, 1.5],
    cmap="plasma",
    show_scalar_bar=False,
)
pl.open_gif("barth_sextic.gif")

for angle in np.linspace(0, np.pi, 15)[:-1]:
    new_mesh = grid.contour(
        1, values, method="flying_edges", rng=[angle_to_range(angle), 0]
    )
    mesh.overwrite(new_mesh)
    pl.update_scalars(np.linalg.norm(new_mesh.points, axis=1), render=False)
    pl.write_frame()

pl.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Background Image
================

Add a background image with
`pyvista.Plotter.add_background_image`.


In [ ]:
import pyvista as pv
from pyvista import examples

Plot an airplane with the map of the earth in the background


In [ ]:
earth_alt = examples.download_topo_global()

pl = pv.Plotter()
actor = pl.add_mesh(examples.load_airplane(), smooth_shading=True)
pl.add_background_image(examples.mapfile)
pl.show()

Plot several earth related plots


In [ ]:
pl = pv.Plotter(shape=(2, 2))

pl.subplot(0, 0)
pl.add_text("Earth Visible as Map")
pl.add_background_image(examples.mapfile, as_global=False)

pl.subplot(0, 1)
pl.add_text("Earth Altitude")
actor = pl.add_mesh(earth_alt, cmap="gist_earth")

pl.subplot(1, 0)
topo = examples.download_topo_land()
actor = pl.add_mesh(topo, cmap="gist_earth")
pl.add_text("Earth Land Altitude")

pl.subplot(1, 1)
pl.add_text("Earth Visible as Globe")
pl.add_mesh(examples.load_globe(), smooth_shading=True)

pl.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Colormap Choices
================

Use a Matplotlib, Colorcet, cmocean, or custom colormap when plotting
scalar values.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv
from matplotlib.colors import ListedColormap
from pyvista import examples

Any colormap built for `matplotlib`, `colorcet`, or `cmocean` is fully
compatible with PyVista. Colormaps are typically specified by passing
the string name of the colormap to the plotting routine via the `cmap`
argument.

See [Matplotlib\'s complete list of available
colormaps](https://matplotlib.org/tutorials/colors/colormaps.html),
[Colorcet\'s complete
list](https://colorcet.holoviz.org/user_guide/index.html), and
[cmocean\'s complete list](https://matplotlib.org/cmocean/).


Custom Made Colormaps
=====================

To get started using a custom colormap, download some data with scalar
values to plot.


In [ ]:
mesh = examples.download_st_helens().warp_by_scalar()
# Add scalar array with range (0, 100) that correlates with elevation
mesh["values"] = pv.plotting.normalize(mesh["Elevation"]) * 100

Build a custom colormap - here we make a colormap with 5 discrete colors
and we specify the ranges where those colors fall:


In [ ]:
# Define the colors we want to use
blue = np.array([12 / 256, 238 / 256, 246 / 256, 1.0])
black = np.array([11 / 256, 11 / 256, 11 / 256, 1.0])
grey = np.array([189 / 256, 189 / 256, 189 / 256, 1.0])
yellow = np.array([255 / 256, 247 / 256, 0 / 256, 1.0])
red = np.array([1.0, 0.0, 0.0, 1.0])

mapping = np.linspace(mesh["values"].min(), mesh["values"].max(), 256)
newcolors = np.empty((256, 4))
newcolors[mapping >= 80] = red
newcolors[mapping < 80] = grey
newcolors[mapping < 55] = yellow
newcolors[mapping < 30] = blue
newcolors[mapping < 1] = black

# Make the colormap from the listed colors
my_colormap = ListedColormap(newcolors)

Simply pass the colormap to the plotting routine!


In [ ]:
mesh.plot(scalars="values", cmap=my_colormap)

Or you could make a simple colormap\... any Matplotlib colormap can be
passed to PyVista!


In [ ]:
boring_cmap = plt.cm.get_cmap("viridis", 5)
mesh.plot(scalars="values", cmap=boring_cmap)

You can also pass a list of color strings to the color map. This
approach divides up the colormap into 5 equal parts.


In [ ]:
mesh.plot(scalars=mesh["values"], cmap=["black", "blue", "yellow", "grey", "red"])

If you still wish to have control of the separation of values, you can
do this by creating a scalar array and passing that to the plotter along
with the the colormap


In [ ]:
scalars = np.empty(mesh.n_points)
scalars[mesh["values"] >= 80] = 4  # red
scalars[mesh["values"] < 80] = 3  # grey
scalars[mesh["values"] < 55] = 2  # yellow
scalars[mesh["values"] < 30] = 1  # blue
scalars[mesh["values"] < 1] = 0  # black

mesh.plot(scalars=scalars, cmap=["black", "blue", "yellow", "grey", "red"])

Matplotlib vs. Colorcet
=======================

Let\'s compare Colorcet\'s perceptually uniform \"fire\" colormap to
Matplotlib\'s \"hot\" colormap much like the example on the [first page
of Colorcet\'s docs](https://colorcet.holoviz.org/index.html).

The \"hot\" version washes out detail at the high end, as if the image
is overexposed, while \"fire\" makes detail visible throughout the data
range.

Please note that in order to use Colorcet\'s colormaps including
\"fire\", you must have Colorcet installed in your Python environment:
`pip install colorcet`


In [ ]:
p = pv.Plotter(shape=(2, 2), border=False)
p.subplot(0, 0)
p.add_mesh(
    mesh,
    scalars="Elevation",
    cmap="fire",
    lighting=True,
    scalar_bar_args={"title": "Colorcet Fire"},
)

p.subplot(0, 1)
p.add_mesh(
    mesh,
    scalars="Elevation",
    cmap="fire",
    lighting=False,
    scalar_bar_args={"title": "Colorcet Fire (No Lighting)"},
)

p.subplot(1, 0)
p.add_mesh(
    mesh,
    scalars="Elevation",
    cmap="hot",
    lighting=True,
    scalar_bar_args={"title": "Matplotlib Hot"},
)

p.subplot(1, 1)
p.add_mesh(
    mesh,
    scalars="Elevation",
    cmap="hot",
    lighting=False,
    scalar_bar_args={"title": "Matplotlib Hot (No Lighting)"},
)

p.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Marching Cubes
==============

Generate a surface from a scalar field using the flying edges and
marching cubes filters as provided by the `contour
<pyvista.DataSetFilters.contour>`{.interpreted-text role="func"} filter.

Special thanks to GitHub user [stla](https://gist.github.com/stla) for
providing examples.


In [ ]:
import numpy as np
import pyvista as pv

Spider Cage
===========

Use the marching cubes algorithm to extract the isosurface generated
from the spider cage function.


In [ ]:
a = 0.9


def spider_cage(x, y, z):
    x2 = x * x
    y2 = y * y
    x2_y2 = x2 + y2
    return (np.sqrt((x2 - y2) ** 2 / x2_y2 + 3 * (z * np.sin(a)) ** 2) - 3) ** 2 + 6 * (
        np.sqrt((x * y) ** 2 / x2_y2 + (z * np.cos(a)) ** 2) - 1.5
    ) ** 2


# create a uniform grid to sample the function with
n = 100
x_min, y_min, z_min = -5, -5, -3
grid = pv.UniformGrid(
    dims=(n, n, n),
    spacing=(abs(x_min) / n * 2, abs(y_min) / n * 2, abs(z_min) / n * 2),
    origin=(x_min, y_min, z_min),
)
x, y, z = grid.points.T

# sample and plot
values = spider_cage(x, y, z)
mesh = grid.contour(1, values, method="marching_cubes", rng=[1, 0])
dist = np.linalg.norm(mesh.points, axis=1)
mesh.plot(
    scalars=dist, smooth_shading=True, specular=5, cmap="plasma", show_scalar_bar=False
)

Barth Sextic
============

Use the flying edges algorithm to extract the isosurface generated from
the Barth sextic function.


In [ ]:
phi = (1 + np.sqrt(5)) / 2
phi2 = phi * phi


def barth_sextic(x, y, z):
    x2 = x * x
    y2 = y * y
    z2 = z * z
    arr = (
        3 * (phi2 * x2 - y2) * (phi2 * y2 - z2) * (phi2 * z2 - x2)
        - (1 + 2 * phi) * (x2 + y2 + z2 - 1) ** 2
    )
    nan_mask = x2 + y2 + z2 > 3.1
    arr[nan_mask] = np.nan
    return arr


# create a uniform grid to sample the function with
n = 100
k = 2.0
x_min, y_min, z_min = -k, -k, -k
grid = pv.UniformGrid(
    dims=(n, n, n),
    spacing=(abs(x_min) / n * 2, abs(y_min) / n * 2, abs(z_min) / n * 2),
    origin=(x_min, y_min, z_min),
)
x, y, z = grid.points.T

# sample and plot
values = barth_sextic(x, y, z)
mesh = grid.contour(1, values, method="flying_edges", rng=[-0.0, 0])
dist = np.linalg.norm(mesh.points, axis=1)
mesh.plot(
    scalars=dist, smooth_shading=True, specular=5, cmap="plasma", show_scalar_bar=False
)

Animate Barth Sextic
====================

Show 15 frames of various isocurves extracted from the Barth sextic
function.


In [ ]:
def angle_to_range(angle):
    return -2 * np.sin(angle)


mesh = grid.contour(1, values, method="flying_edges", rng=[angle_to_range(0), 0])
dist = np.linalg.norm(mesh.points, axis=1)

pl = pv.Plotter()
pl.add_mesh(
    mesh,
    scalars=dist,
    smooth_shading=True,
    specular=5,
    rng=[0.5, 1.5],
    cmap="plasma",
    show_scalar_bar=False,
)
pl.open_gif("barth_sextic.gif")

for angle in np.linspace(0, np.pi, 15)[:-1]:
    new_mesh = grid.contour(
        1, values, method="flying_edges", rng=[angle_to_range(angle), 0]
    )
    mesh.overwrite(new_mesh)
    pl.update_scalars(np.linalg.norm(new_mesh.points, axis=1), render=False)
    pl.write_frame()

pl.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Create a GIF Movie
==================

Generate a moving gif from an active plotter.

Use `lighting=False` to reduce the size of the color space to avoid
\"jittery\" GIFs, especially for the scalar bar.


In [ ]:
import numpy as np
import pyvista as pv

x = np.arange(-10, 10, 0.5)
y = np.arange(-10, 10, 0.5)
x, y = np.meshgrid(x, y)
r = np.sqrt(x**2 + y**2)
z = np.sin(r)

# Create and structured surface
grid = pv.StructuredGrid(x, y, z)

# Create a plotter object and set the scalars to the Z height
plotter = pv.Plotter(notebook=False, off_screen=True)
plotter.add_mesh(
    grid,
    scalars=z.ravel(),
    lighting=False,
    show_edges=True,
    scalar_bar_args={"title": "Height"},
    clim=[-1, 1],
)

# Open a gif
plotter.open_gif("wave.gif")

pts = grid.points.copy()

# Update Z and write a frame for each updated position
nframe = 15
for phase in np.linspace(0, 2 * np.pi, nframe + 1)[:nframe]:
    z = np.sin(r + phase)
    pts[:, -1] = z.ravel()
    plotter.update_coordinates(pts, render=False)
    plotter.update_scalars(z.ravel(), render=False)

    # Write a frame. This triggers a render.
    plotter.write_frame()

# Closes and finalizes movie
plotter.close()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Linked Views in Subplots
========================


In [ ]:
import numpy as np
import pyvista as pv
from pyvista import examples

pv.set_plot_theme("document")

# download mesh
mesh = examples.download_cow()

decimated = mesh.decimate_boundary(target_reduction=0.75)

p = pv.Plotter(shape=(1, 2), border=False)
p.subplot(0, 0)
p.add_text("Original mesh", font_size=24)
p.add_mesh(mesh, show_edges=True, color=True)
p.subplot(0, 1)
p.add_text("Decimated version", font_size=24)
p.add_mesh(decimated, color=True, show_edges=True)

p.link_views()  # link all the views
# Set a camera position to all linked views
p.camera_position = [(15, 5, 0), (0, 0, 0), (0, 1, 0)]

p.open_gif("linked.gif")
# Update camera and write a frame for each updated position
nframe = 15
for i in range(nframe):
    p.camera_position = [
        (15 * np.cos(i * np.pi / 45.0), 5.0, 15 * np.sin(i * np.pi / 45.0)),
        (0, 0, 0),
        (0, 1, 0),
    ]
    p.write_frame()

# Close movie and delete object
p.close()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Create a MP4 Movie
==================

Create an animated MP4 movie of a rendering scene.

This movie will appear static since MP4 movies will not be rendered on a
sphinx gallery example.


In [ ]:
import numpy as np
import pyvista as pv

filename = "sphere-shrinking.mp4"

mesh = pv.Sphere()
mesh.cell_data["data"] = np.random.random(mesh.n_cells)

plotter = pv.Plotter()
# Open a movie file
plotter.open_movie(filename)

# Add initial mesh
plotter.add_mesh(mesh, scalars="data", clim=[0, 1])
# Add outline for shrinking reference
plotter.add_mesh(mesh.outline_corners())

plotter.show(auto_close=False)  # only necessary for an off-screen movie

# Run through each frame
plotter.write_frame()  # write initial data

# Update scalars on each frame
for i in range(100):
    random_points = np.random.random(mesh.points.shape)
    mesh.points = random_points * 0.01 + mesh.points * 0.99
    mesh.points -= mesh.points.mean(0)
    mesh.cell_data["data"] = np.random.random(mesh.n_cells)
    plotter.add_text(f"Iteration: {i}", name="time-label")
    plotter.write_frame()  # Write this frame

# Be sure to close the plotter when finished
plotter.close()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Create a GIF Movie of a Static Object with a Moving Colormap
============================================================

Generate a gif movie of a Hopf torus with a moving colormap, by updating
the scalars.


In [ ]:
import numpy as np
import pyvista as pv


# A spherical curve
def scurve(t):
    alpha = np.pi / 2 - (np.pi / 2 - 0.44) * np.cos(3 * t)
    beta = t + 0.44 * np.sin(6 * t)
    return np.array(
        [np.sin(alpha) * np.cos(beta), np.sin(alpha) * np.sin(beta), np.cos(alpha)]
    )


# Hopf fiber
def hopf_fiber(p, phi):
    return np.array(
        [
            (1 + p[2]) * np.cos(phi),
            p[0] * np.sin(phi) - p[1] * np.cos(phi),
            p[0] * np.cos(phi) + p[1] * np.sin(phi),
            (1 + p[2]) * np.sin(phi),
        ]
    ) / np.sqrt(2 * (1 + p[2]))


# Stereographic projection
def stereo_proj(q):
    return q[0:3] / (1 - q[3])


# Parameterization of the Hopf torus
def hopf_torus(t, phi):
    return stereo_proj(hopf_fiber(scurve(t), phi))


# Create the mesh
angle_u = np.linspace(-np.pi, np.pi, 400)
angle_v = np.linspace(0, np.pi, 200)
u, v = np.meshgrid(angle_u, angle_v)
x, y, z = hopf_torus(u, v)
grid = pv.StructuredGrid(x, y, z)
mesh = grid.extract_geometry().clean(tolerance=1e-6)

# Distances normalized to [0, 2*pi]
dists = np.linalg.norm(mesh.points, axis=1)
dists = 2 * np.pi * (dists - dists.min()) / (dists.max() - dists.min())

# Make the movie
pltr = pv.Plotter(window_size=[512, 512])
pltr.set_focus([0, 0, 0])
pltr.set_position([40, 0, 0])
pltr.add_mesh(
    mesh,
    scalars=np.sin(dists),
    smooth_shading=True,
    specular=10,
    cmap="nipy_spectral",
    show_scalar_bar=False,
)
pltr.open_gif("Hopf_torus.gif")

for t in np.linspace(0, 2 * np.pi, 60, endpoint=False):
    pltr.update_scalars(np.sin(dists - t), render=False)
    pltr.write_frame()

pltr.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Orbiting
========

Orbit around a scene.

The quality of the movie will be better when using
`p.open_movie('orbit.mp4')` instead of `p.open_gif('orbit.gif')`

For orbiting to work you first have to show the scene and leave the
plotter open with `.show(auto_close=False)`. You may also have to set
`pv.Plotter(off_screen=True)`

Use `lighting=False` to reduce the size of the color space to avoid
\"jittery\" GIFs when showing the scalar bar.


In [ ]:
import pyvista as pv
from pyvista import examples

mesh = examples.download_st_helens().warp_by_scalar()

Orbit around the Mt. St Helens dataset.


In [ ]:
p = pv.Plotter()
p.add_mesh(mesh, lighting=False)
p.camera.zoom(1.5)
p.show(auto_close=False)
path = p.generate_orbital_path(n_points=36, shift=mesh.length)
p.open_gif("orbit.gif")
p.orbit_on_path(path, write_frames=True)
p.close()

In [ ]:
p = pv.Plotter()
p.add_mesh(mesh, lighting=False)
p.show_grid()
p.show(auto_close=False)
viewup = [0.5, 0.5, 1]
path = p.generate_orbital_path(factor=2.0, shift=10000, viewup=viewup, n_points=36)
p.open_gif("orbit.gif")
p.orbit_on_path(path, write_frames=True, viewup=[0, 0, 1], step=0.05)
p.close()

In [ ]:
mesh = examples.download_dragon()
viewup = [0, 1, 0]

In [ ]:
p = pv.Plotter()
p.add_mesh(mesh)
p.show(auto_close=False)
path = p.generate_orbital_path(factor=2.0, n_points=36, viewup=viewup, shift=0.2)
p.open_gif("orbit.gif")
p.orbit_on_path(path, write_frames=True, viewup=viewup, step=0.05)
p.close()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Customize Scalar Bars
=====================

Walk through of all the different capabilities of scalar bars and how a
user can customize scalar bars.


In [ ]:
import pyvista as pv
from pyvista import examples

By default, when plotting a dataset with a scalar array, a scalar bar
for that array is added. To turn off this behavior, a user could specify
`show_scalar_bar=False` when calling `.add_mesh()`. Let\'s start with a
sample dataset provide via PyVista to demonstrate the default behavior
of scalar bar plotting:


In [ ]:
# Load St Helens DEM and warp the topography
mesh = examples.download_st_helens().warp_by_scalar()

# First a default plot with jet colormap
p = pv.Plotter()
# Add the data, use active scalar for coloring, and show the scalar bar
p.add_mesh(mesh)
# Display the scene
p.show()

We could also plot the scene with an interactive scalar bar to move
around and place where we like by specifying passing keyword arguments
to control the scalar bar via the `scalar_bar_args` parameter in
`pyvista.BasePlotter.add_mesh`. The
keyword arguments to control the scalar bar are defined in
`pyvista.BasePlotter.add_scalar_bar`.


In [ ]:
# create dictionary of parameters to control scalar bar
sargs = dict(interactive=True)  # Simply make the bar interactive

p = pv.Plotter(notebook=False)  # If in IPython, be sure to show the scene
p.add_mesh(mesh, scalar_bar_args=sargs)
p.show()
# Remove from plotters so output is not produced in docs
pv.plotting._ALL_PLOTTERS.clear()

![](https://docs.pyvista.org/_images/scalar-bar-interactive.gif)

Or manually define the scalar bar\'s location:


In [ ]:
# Set a custom position and size
sargs = dict(height=0.25, vertical=True, position_x=0.05, position_y=0.05)

p = pv.Plotter()
p.add_mesh(mesh, scalar_bar_args=sargs)
p.show()

The text properties of the scalar bar can also be controlled:


In [ ]:
# Controlling the text properties
sargs = dict(
    title_font_size=20,
    label_font_size=16,
    shadow=True,
    n_labels=3,
    italic=True,
    fmt="%.1f",
    font_family="arial",
)

p = pv.Plotter()
p.add_mesh(mesh, scalar_bar_args=sargs)
p.show()

Labelling values outside of the scalar range


In [ ]:
p = pv.Plotter()
p.add_mesh(
    mesh,
    clim=[1000, 2000],
    below_color="blue",
    above_color="red",
    scalar_bar_args=sargs,
)
p.show()

Annotate values of interest using a dictionary. The key of the
dictionary must be the value to annotate, and the value must be the
string label.


In [ ]:
# Make a dictionary for the annotations
annotations = {
    2300: "High",
    805.3: "Cutoff value",
}

p = pv.Plotter()
p.add_mesh(mesh, scalars="Elevation", annotations=annotations)
p.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Saving Screenshots
==================


In [ ]:
import matplotlib.pyplot as plt
import pyvista as pv
from pyvista import examples

# Get a sample file
filename = examples.planefile
mesh = pv.read(filename)

You can also take a screenshot without creating an interactive plot
window using the `pyvista.Plotter`:


In [ ]:
plotter = pv.Plotter(off_screen=True)
plotter.add_mesh(mesh, color="orange")
plotter.show(screenshot="airplane.png")

The `img` array can be used to plot the screenshot in `matplotlib`:


In [ ]:
plt.imshow(plotter.image)
plt.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Applying Textures
=================

Plot a mesh with an image projected onto it as a texture.


In [ ]:
import numpy as np
import pyvista as pv
from matplotlib.cm import get_cmap
from pyvista import examples

Texture mapping is easily implemented using PyVista. Many of the
geometric objects come preloaded with texture coordinates, so quickly
creating a surface and displaying an image is simply:


In [ ]:
# load a sample texture
tex = examples.download_masonry_texture()

# create a surface to host this texture
surf = pv.Cylinder()

surf.plot(texture=tex)

But what if your dataset doesn\'t have texture coordinates? Then you can
harness the
`pyvista.DataSetFilters.texture_map_to_plane` filter to properly map an image to a dataset\'s surface.
For example, let\'s map that same image of bricks to a curvey surface:


In [ ]:
# create a structured surface
x = np.arange(-10, 10, 0.25)
y = np.arange(-10, 10, 0.25)
x, y = np.meshgrid(x, y)
r = np.sqrt(x**2 + y**2)
z = np.sin(r)
curvsurf = pv.StructuredGrid(x, y, z)

# Map the curved surface to a plane - use best fitting plane
curvsurf.texture_map_to_plane(inplace=True)

curvsurf.plot(texture=tex)

Display scalar data along with a texture by ensuring the
`interpolate_before_map` setting is `False` and specifying both the
`texture` and `scalars` arguments.


In [ ]:
elevated = curvsurf.elevation()

elevated.plot(
    scalars="Elevation", cmap="terrain", texture=tex, interpolate_before_map=False
)

Note that this process can be completed with any image texture!


In [ ]:
# use the puppy image
tex = examples.download_puppy_texture()
curvsurf.plot(texture=tex)

Textures from Files
===================

What about loading your own texture from an image? This is often most
easily done using the `pyvista.read_texture` function - simply pass an image file\'s path, and this
function with handle making a `vtkTexture` for you to use.


In [ ]:
image_file = examples.mapfile
tex = pv.read_texture(image_file)
curvsurf.plot(texture=tex)

NumPy Arrays as Textures
========================

Want to use a programmatically built image?
`pyvista.UniformGrid` objects can be
converted to textures using `pyvista.image_to_texture` and 3D NumPy (X by Y by RGB) arrays can be converted to
textures using `pyvista.numpy_to_texture`.


In [ ]:
# create an image using numpy,
xx, yy = np.meshgrid(np.linspace(-200, 200, 20), np.linspace(-200, 200, 20))
A, b = 500, 100
zz = A * np.exp(-0.5 * ((xx / b) ** 2.0 + (yy / b) ** 2.0))

# Creating a custom RGB image
cmap = get_cmap("nipy_spectral")
norm = lambda x: (x - np.nanmin(x)) / (np.nanmax(x) - np.nanmin(x))
hue = norm(zz.ravel())
colors = (cmap(hue)[:, 0:3] * 255.0).astype(np.uint8)
image = colors.reshape((xx.shape[0], xx.shape[1], 3), order="F")

# Convert 3D numpy array to texture
tex = pv.numpy_to_texture(image)

# Render it!
curvsurf.plot(texture=tex)

Create a GIF Movie with updating textures
=========================================

Generate a moving gif from an active plotter with updating textures.


In [ ]:
# Create a plotter object
plotter = pv.Plotter(notebook=False, off_screen=True)

# Open a gif
plotter.open_gif("texture.gif")

pts = curvsurf.points.copy()

# Update Z and write a frame for each updated position
nframe = 15
for phase in np.linspace(0, 2 * np.pi, nframe + 1)[:nframe]:

    # create an image using numpy,
    z = np.sin(r + phase)
    pts[:, -1] = z.ravel()

    # Creating a custom RGB image
    zz = A * np.exp(-0.5 * ((xx / b) ** 2.0 + (yy / b) ** 2.0))
    hue = norm(zz.ravel()) * 0.5 * (1.0 + np.sin(phase))
    colors = (cmap(hue)[:, 0:3] * 255.0).astype(np.uint8)
    image = colors.reshape((xx.shape[0], xx.shape[1], 3), order="F")

    # Convert 3D numpy array to texture
    tex = pv.numpy_to_texture(image)

    plotter.add_mesh(curvsurf, smooth_shading=True, texture=tex)
    plotter.update_coordinates(pts, render=False)

    # must update normals when smooth shading is enabled
    plotter.mesh.compute_normals(cell_normals=False, inplace=True)
    plotter.write_frame()
    plotter.clear()

# Closes and finalizes movie
plotter.close()

Textures with Transparency
==========================

Textures can also specify per-pixel opacity values. The image must
contain a 4th channel specifying the opacity value from 0
\[transparent\] to 255 \[fully visible\]. To enable this feature just
pass the opacity array as the 4th channel of the image as a 3
dimensional matrix with shape \[nrows, ncols, 4\]
`pyvista.numpy_to_texture`.

Here we can download an image that has an alpha channel:


In [ ]:
rgba = examples.download_rgba_texture()
rgba.n_components

In [ ]:
# Render it!
curvsurf.plot(texture=rgba, show_grid=True)

Repeating Textures
==================

What if you have a single texture that you\'d like to repeat across a
mesh? Simply define the texture coordinates for all nodes explicitly.

Here we create the texture coordinates to fill up the grid with several
mappings of a single texture. In order to do this we must define texture
coordinates outside of the typical `(0, 1)` range:


In [ ]:
axial_num_puppies = 4
xc = np.linspace(0, axial_num_puppies, curvsurf.dimensions[0])
yc = np.linspace(0, axial_num_puppies, curvsurf.dimensions[1])

xxc, yyc = np.meshgrid(xc, yc)
puppy_coords = np.c_[yyc.ravel(), xxc.ravel()]

By defining texture coordinates that range `(0, 4)` on each axis, we
will produce 4 repetitions of the same texture on this mesh.

Then we must associate those texture coordinates with the mesh through
the `pyvista.DataSet.active_t_coords`
property.


In [ ]:
curvsurf.active_t_coords = puppy_coords

Now display all the puppies!


In [ ]:
# use the puppy image
tex = examples.download_puppy_texture()
curvsurf.plot(texture=tex, cpos="xy")

Spherical Texture Coordinates
=============================

We have a built in convienance method for mapping textures to spherical
coordinate systems much like the planar mapping demoed above.


In [ ]:
mesh = pv.Sphere()
tex = examples.download_masonry_texture()

mesh.texture_map_to_sphere(inplace=True)
mesh.plot(texture=tex)

The helper method above does not always produce the desired texture
coordinates, so sometimes it must be done manually. Here is a great,
user contributed example from [this support
issue](https://github.com/pyvista/pyvista-support/issues/257)

Manually create the texture coordinates for a globe map. First, we
create the mesh that will be used as the globe. Note the
`start\_theta` for a slight overlappig


In [ ]:
sphere = pv.Sphere(
    radius=1,
    theta_resolution=120,
    phi_resolution=120,
    start_theta=270.001,
    end_theta=270,
)

# Initialize the texture coordinates array
sphere.active_t_coords = np.zeros((sphere.points.shape[0], 2))

# Populate by manually calculating
for i in range(sphere.points.shape[0]):
    sphere.active_t_coords[i] = [
        0.5 + np.arctan2(-sphere.points[i, 0], sphere.points[i, 1]) / (2 * np.pi),
        0.5 + np.arcsin(sphere.points[i, 2]) / np.pi,
    ]

# And let's display it with a world map
tex = examples.load_globe_texture()
sphere.plot(texture=tex)

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Control Global and Local Plotting Themes
========================================

PyVista allows you to set global and local plotting themes to easily set
default plotting parameters.


In [ ]:
import pyvista as pv
from pyvista import examples

Define a simple plotting routine for comparing the themes.


In [ ]:
mesh = examples.download_st_helens().warp_by_scalar()


def plot_example():
    p = pv.Plotter()
    p.add_mesh(mesh)
    p.add_bounding_box()
    p.show()

PyVista\'s default color theme is chosen to be generally easy on your
eyes and is best used when working long hours on your visualization
project. The grey background and warm colormaps are chosen to make sure
3D renderings do not drastically change the brightness of your screen
when working in dark environments.

Here\'s an example of our default plotting theme - this is what you
would see by default after running any of our examples locally.


In [ ]:
pv.set_plot_theme("default")
plot_example()

PyVista also ships with a few plotting themes:

-   `'ParaView'`: this is designed to mimic ParaView\'s default plotting
    theme.
-   `'dark'`: this is designed to be night-mode friendly with dark
    backgrounds and color schemes.
-   `'document'`: this is built for use in document style plotting and
    making publication quality figures.


Demo the `'ParaView'` theme.


In [ ]:
pv.set_plot_theme("paraview")

plot_example()

Demo the `'dark'` theme.


In [ ]:
pv.set_plot_theme("dark")

plot_example()

Demo the `'document'` theme. This theme is used on our online examples.


In [ ]:
pv.set_plot_theme("document")

plot_example()

Note that you can also use color gradients for the background of the
plotting window!


In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(mesh)
plotter.show_grid()
# Here we set the gradient
plotter.set_background("royalblue", top="aliceblue")
cpos = plotter.show()

Modifying the Global Theme
==========================

You can control how meshes are displayed by setting individual
parameters when plotting like `mesh.plot(show_edges=True)`, or by
setting a global theme. You can also control individual parameters how
all meshes are displayed by default via `pyvista.global_theme`.

Here, we print out the current global defaults for all `pyvista` meshes.
These values have been changed by the previous \"Document\" theme.


In [ ]:
pv.global_theme

By default, edges are not shown on meshes unless explicitly specified
when plotting a mesh via `show_edges=True`. You can change this default
behavior globally by changing the default parameter.


In [ ]:
pv.global_theme.show_edges = True
cpos = pv.Sphere().plot()

You can reset pyvista to default behavior with `restore_defaults`. Note
that the figure\'s color was reset to the default \"white\" color rather
than the \"tan\" color default with the document theme. Under the hood,
each theme applied changes the global plot defaults stored within
`pyvista.global_theme.`


In [ ]:
pv.global_theme.restore_defaults()
cpos = pv.Sphere().plot()

Creating a Custom Theme and Applying it Globally
================================================

You can create a custom theme by modifying one of the existing themes
and then loading it into the global plotting defaults.

Here, we create a dark theme that plots meshes red by default while
showing edges.


In [ ]:
from pyvista import themes

my_theme = themes.DarkTheme()
my_theme.color = "red"
my_theme.lighting = False
my_theme.show_edges = True
my_theme.axes.box = True

pv.global_theme.load_theme(my_theme)
cpos = pv.Sphere().plot()

Creating a Custom Theme and Applying it to a Single Plotter
===========================================================

In this example, we create a custom theme from the base \"default\"
theme and then apply it to a single plotter. Note that this does not
change the behavior of the global \"defaults\", which are still set to
the modified `DarkTheme`.

This approach carries the advantage that you can maintain several themes
and apply them to one or more plotters.


In [ ]:
from pyvista import themes

my_theme = themes.DefaultTheme()
my_theme.color = "black"
my_theme.lighting = True
my_theme.show_edges = True
my_theme.edge_color = "white"
my_theme.background = "white"

cpos = pv.Sphere().plot(theme=my_theme)

Alternatively, set the theme of an instance of `Plotter`.


In [ ]:
pl = pv.Plotter(theme=my_theme)
# pl.theme = my_theme  # alternatively use the setter
pl.add_mesh(pv.Cube())
cpos = pl.show()

Reset to use the document theme


In [ ]:
pv.set_plot_theme("document")

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Plot Vector Component
=====================

Plot a single component of a vector as a scalar array.

We can plot individual components of multi-component arrays with the
`component` argument of the `add_mesh` method.


In [ ]:
import pyvista as pv
from pyvista import examples

Download an example notched beam stress


In [ ]:
mesh = examples.download_notch_displacement()

The default behavior with no component specified is to use the vector
magnitude. We can access each component by specifying the component
argument.


In [ ]:
dargs = dict(
    scalars="Nodal Displacement",
    cmap="jet",
    show_scalar_bar=False,
)

pl = pv.Plotter(shape=(2, 2))
pl.subplot(0, 0)
pl.add_mesh(mesh, **dargs)
pl.add_text("Normalized Displacement", color="k")
pl.subplot(0, 1)
pl.add_mesh(mesh.copy(), component=0, **dargs)
pl.add_text("X Displacement", color="k")
pl.subplot(1, 0)
pl.add_mesh(mesh.copy(), component=1, **dargs)
pl.add_text("Y Displacement", color="k")
pl.subplot(1, 1)
pl.add_mesh(mesh.copy(), component=2, **dargs)
pl.add_text("Z Displacement", color="k")
pl.link_views()
pl.camera_position = "iso"
pl.background_color = "white"
pl.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

Volume Rendering
================

Volume render uniform mesh types like
`pyvista.UniformGrid` or 3D NumPy
arrays.

This also explores how to extract a volume of interest (VOI) from a
`pyvista.UniformGrid` using the
`pyvista.UniformGridFilters.extract_subset` filter.


In [ ]:
import pyvista as pv
from pyvista import examples

# Download a volumetric dataset
vol = examples.download_knee_full()
vol

Simple Volume Render
====================


In [ ]:
# A nice camera position
cpos = [(-381.74, -46.02, 216.54), (74.8305, 89.2905, 100.0), (0.23, 0.072, 0.97)]

vol.plot(volume=True, cmap="bone", cpos=cpos)

Opacity Mappings
================

Or use the `pyvista.BasePlotter.add_volume` method like below. Note that here we use a non-default
opacity mapping to a sigmoid:


In [ ]:
p = pv.Plotter()
p.add_volume(vol, cmap="bone", opacity="sigmoid")
p.camera_position = cpos
p.show()

You can also use a custom opacity mapping


In [ ]:
opacity = [0, 0, 0, 0.1, 0.3, 0.6, 1]

p = pv.Plotter()
p.add_volume(vol, cmap="viridis", opacity=opacity)
p.camera_position = cpos
p.show()

We can also use a shading technique when volume rendering with the
`shade` option


In [ ]:
p = pv.Plotter(shape=(1, 2))
p.add_volume(vol, cmap="viridis", opacity=opacity, shade=False)
p.add_text("No shading")
p.subplot(0, 1)
p.add_volume(vol, cmap="viridis", opacity=opacity, shade=True)
p.add_text("Shading")
p.link_views()
p.camera_position = cpos
p.show()

Cool Volume Examples
====================

Here are a few more cool volume rendering examples


In [ ]:
head = examples.download_head()

p = pv.Plotter()
p.add_volume(head, cmap="cool", opacity="sigmoid_6")
p.camera_position = [(-228.0, -418.0, -158.0), (94.0, 122.0, 82.0), (-0.2, -0.3, 0.9)]
p.show()

In [ ]:
bolt_nut = examples.download_bolt_nut()

p = pv.Plotter()
p.add_volume(bolt_nut, cmap="coolwarm", opacity="sigmoid_5")
p.show()

In [ ]:
frog = examples.download_frog()

p = pv.Plotter()
p.add_volume(frog, cmap="viridis", opacity="sigmoid_6")
p.camera_position = [
    (929.0, 1067.0, -278.9),
    (249.5, 234.5, 101.25),
    (-0.2048, -0.2632, -0.9427),
]
p.show()

Extracting a VOI
================

Use the `pyvista.UniformGridFilters.extract_subset` filter to extract a volume of interest/subset volume to
volume render. This is ideal when dealing with particularly large
volumes and you want to volume render only a specific region.


In [ ]:
# Load a particularly large volume
large_vol = examples.download_damavand_volcano()
large_vol

In [ ]:
opacity = [0, 0.75, 0, 0.75, 1.0]
clim = [0, 100]

p = pv.Plotter()
p.add_volume(
    large_vol,
    cmap="magma",
    clim=clim,
    opacity=opacity,
    opacity_unit_distance=6000,
)
p.show()

Woah, that\'s a big volume! We probably don\'t want to volume render the
whole thing. So let\'s extract a region of interest under the volcano.

The region we will extract will be between nodes 175 and 200 on the
x-axis, between nodes 105 and 132 on the y-axis, and between nodes 98
and 170 on the z-axis.


In [ ]:
voi = large_vol.extract_subset([175, 200, 105, 132, 98, 170])

p = pv.Plotter()
p.add_mesh(large_vol.outline(), color="k")
p.add_mesh(voi, cmap="magma")
p.show()

Ah, much better. Let\'s now volume render that region of interest!


In [ ]:
p = pv.Plotter()
p.add_volume(voi, cmap="magma", clim=clim, opacity=opacity, opacity_unit_distance=2000)
p.camera_position = [
    (531554.5542909054, 3944331.800171338, 26563.04809259223),
    (599088.1433822059, 3982089.287834022, -11965.14728669936),
    (0.3738545892415734, 0.244312810377319, 0.8947312427698892),
]
p.show()

### Exercise

Let's create an animation that varies the radius of the sphere from 0.0 to 0.2.

In [ ]:
# Your codes here.

In [ ]:
import pyvista as pv

# Create a plotter object
plotter = pv.Plotter()

# Open a gif
plotter.open_gif("sphere.gif")
mesh = pv.Sphere()
plotter.add_mesh(mesh, show_edges=True)
pts = mesh.points.copy()

# Update radius and write a frame for each updated position
nframe = 100
for radius in np.linspace(0, 0.2, nframe + 1):
    plotter.update_coordinates(radius * pts, mesh, render=True)
    plotter.write_frame()

# Closes and finalizes movie
plotter.close()